[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1lbil2FwHB3Fcv8CL5F2sSi2nxXk_toQ5#scrollTo=HPrAc0uaVSrx)
## Step 1: Install Dependencies

This cell uses `pip` to install the necessary Python packages for the project.
* `flotorch[adk]`: Installs the Flotorch Agent Development Kit (ADK), which provides the core framework for building and running the agent.
* `PyPDF2`: A library used to read and extract text from PDF files.
* `sentence-transformers`: A library used to generate vector embeddings (numerical representations) of text. This is essential for semantic search.
* `faiss-cpu`: Facebook AI Similarity Search (FAISS) library. It's used to create an efficient, local vector index for storing and searching the text embeddings.
* `requests`: A library for making HTTP requests, used here to download the PDF from a URL.
* `numpy`: A fundamental package for numerical operations in Python, required by FAISS and sentence-transformers.

In [ ]:
%pip install flotorch[adk]
%pip install PyPDF2 sentence-transformers faiss-cpu requests numpy

### Step 2: Configuration Variables

This cell defines the constant variables required to configure and authenticate the Flotorch agent.

* `FLOTORCH_API_KEY`: Replace the API key with your actual FloTorch API Key (Check [Prerequisites.ipynb](https://github.com/FloTorch/Resources/blob/main/examples/flotorch-rag-notebooks/faiss-example/01_Prerequisites.ipynb) on how to get the key)

                        Using FloTorch helps you monitor the agentic flow through the FloTorch gateway.
* `FLOTORCH_BASE_URL`: The API gateway endpoint.
* `AGENT_NAME`: A unique name for this specific agent instance, indicating it uses MCP (Multi-turn Conversation Protocol) tools.
* `APP_NAME`: A name for your application, used to group sessions.
* `USER_ID`: A unique identifier for the end-user interacting with the agent.

In [ ]:
from getpass import getpass

FLOTORCH_API_KEY = getpass("FloTorch API Key") # SET YOUR FLOTORCH API KEY HERE
FLOTORCH_BASE_URL = "https://gateway.flotorch.cloud"
AGENT_NAME = "rag-agent"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

## Step 3: Import Core ADK Libraries

This cell imports the primary classes needed from the Flotorch and Google ADK libraries to construct the agent.
* `FlotorchADKAgent`: The main class for creating a Flotorch agent.
* `FlotorchMemoryService`: (Imported but not used) A class for managing persistent memory.
* `FlotorchADKSession`: The class used to manage conversation sessions.
* `Runner`: The ADK class responsible for executing the agent's logic.
* `types`: Google GenAI types, used for structuring messages (e.g., `Part`, `Content`).
* `FunctionTool`: A class used to wrap a standard Python function so it can be used as a "tool" by the LLM agent.

In [ ]:
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.memory import FlotorchMemoryService
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types
from google.adk.tools import FunctionTool

print("Imported necessary libraries successfully")

## Step 4: Define the RAG Tool (Local Vector Store)

This is the most complex cell and contains the core logic for the Retrieval-Augmented Generation (RAG) pipeline.

1.  **Imports**: It imports necessary helper libraries (`os`, `json`, `tempfile`, `faiss`, etc.).
2.  **Helper Functions**:
    * `extract_text_from_pdf(file_path)`: Opens a PDF file, reads each page, and extracts the text.
    * `split_text_into_sentences(text, max_len=300)`: A simple text chunker that splits the extracted text into smaller, manageable chunks (up to 300 characters).
    * `create_faiss_index(sentences, embedder)`: Takes the text chunks, converts them into vector embeddings using the `SentenceTransformer` model, and stores them in a FAISS index.
    * `vector_search(query, embedder, index, sentences, top_k=5)`: Takes a user query, embeds it, and searches the FAISS index for the top 5 most semantically similar text chunks.
3.  **Main Tool Function**:
    * `get_context_local(...)`: This is the main function that orchestrates the RAG process.
    * It first checks if an index (`local_faiss.index`) already exists.
    * If `force_rebuild=True` or no index exists and a PDF is provided (via `pdf_url` or `pdf_path`), it downloads/reads the PDF, extracts text, chunks it, and creates/saves a new FAISS index and a `sentences.json` file.
    * If an index *does* exist (or was just built), it loads the index and sentences from the local files.
    * It then performs a `vector_search` using the user's `query`.
    * Finally, it returns the combined text of the top search results.
4.  **Tool Creation**:
    * `local_vector_tool = FunctionTool(func=get_context_local)`: This line wraps the entire `get_context_local` function into a `FunctionTool` object. This makes the function "callable" by the Flotorch agent, allowing the LLM to decide when to use it to retrieve context.

In [ ]:
from google.adk.tools import FunctionTool
import os
import json
import tempfile
import requests
import faiss
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
from typing import Optional

# PDF text extraction
def extract_text_from_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

# Split text into sentences/chunks
def split_text_into_sentences(text, max_len=300):
    import re
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_len:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Create FAISS index
def create_faiss_index(sentences, embedder):
    embeddings = embedder.encode(sentences, convert_to_numpy=True, normalize_embeddings=True)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index, embeddings

# Vector search
def vector_search(query, embedder, index, sentences, top_k=5):
    query_vec = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, indices = index.search(query_vec, top_k)
    results = []
    for i, score in zip(indices[0], scores[0]):
        results.append({"text": sentences[i], "score": float(score)})
    return results

# Main function
def get_context_local(query: str, pdf_url: str = "", pdf_path: str = "", force_rebuild: bool = False) -> str:
    INDEX_PATH = "local_faiss.index"
    SENTENCES_PATH = "sentences.json"
    MODEL_NAME = "all-MiniLM-L6-v2"
    embedder = SentenceTransformer(MODEL_NAME)

    index_exists = os.path.exists(INDEX_PATH) and os.path.exists(SENTENCES_PATH)

    # Rebuild index if force_rebuild is True OR if PDF provided and index does not exist
    if force_rebuild or ((pdf_path.strip() or pdf_url.strip()) and not index_exists):
        if pdf_path.strip() and os.path.exists(pdf_path):
            print(f"📄 Using local PDF: {pdf_path}")
            text_content = extract_text_from_pdf(pdf_path)
        elif pdf_url.strip():
            print(f"🌐 Downloading PDF from: {pdf_url}")
            temp_pdf = tempfile.mktemp(".pdf")
            open(temp_pdf, "wb").write(requests.get(pdf_url).content)
            text_content = extract_text_from_pdf(temp_pdf)
            os.remove(temp_pdf)
        else:
            return "❌ Invalid PDF path or URL provided."

        if not text_content.strip():
            return "❌ No text extracted from PDF."

        sentences = split_text_into_sentences(text_content)
        index, embeddings = create_faiss_index(sentences, embedder)

        faiss.write_index(index, INDEX_PATH)
        with open(SENTENCES_PATH, "w", encoding="utf-8") as f:
            json.dump(sentences, f, ensure_ascii=False, indent=2)

        print("✅ Index created successfully.")

    # Load existing index
    if not os.path.exists(INDEX_PATH) or not os.path.exists(SENTENCES_PATH):
        return "❌ No local index found. Provide a PDF first (local or URL)."

    index = faiss.read_index(INDEX_PATH)
    with open(SENTENCES_PATH, "r", encoding="utf-8") as f:
        sentences = json.load(f)

    results = vector_search(query, embedder, index, sentences, top_k=5)
    if not results:
        return "⚠️ No relevant results found."

    results.sort(key=lambda x: x["score"], reverse=True)
    return "\n\n".join([r["text"] for r in results])

# Wrap as ADK tool
local_vector_tool = FunctionTool(func=get_context_local)


## Step 5:  Initialize Session Service

This cell initializes the `FlotorchADKSession` service. This service is responsible for handling the creation and retrieval of conversation sessions, using the API key and base URL defined in Cell 2.

In [ ]:
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY, base_url=FLOTORCH_BASE_URL
)

print("Initialized Memory and Session")

## Step 6:  Initialize the Flotorch Agent

This cell creates the agent instance.
* It initializes `FlotorchADKAgent`, passing the agent's name, API key, and base URL.
* Crucially, it passes `custom_tools=[local_vector_tool]`. This tells the agent that it has access to the RAG function we defined in Cell 4. The agent's underlying LLM will be aware of this tool and can choose to call it when it needs to answer a question about a PDF.

In [ ]:
flotorch_client = FlotorchADKAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    custom_tools=[local_vector_tool]
)

agent = flotorch_client.get_agent()
print(f"Advanced FlotorchADKAgent '{agent.name}' created.")

## Step 7:  Configure the Agent Runner and Chat Function

This cell sets up the components needed to run the agent and interact with it.
1.  **Initialize `Runner`**: The `Runner` is initialized, binding the `agent` (from Cell 6) and the `session_service` (from Cell 5) together.
2.  **Define `chat_with_agent`**: This `async` function is a helper to simplify sending messages to the agent.
    * It takes a `query`, `session_id`, and optional `pdf_url` and `force_rebuild` flags.
    * It constructs a message `content` for the agent.
    * It calls `runner.run(...)`, which processes the message. This may involve multiple steps (e.g., LLM calls, tool calls).
    * It iterates through the `events` generated by the runner and returns the text from the `final_response`.

In [ ]:
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

async def chat_with_agent(query, session_id, pdf_url=None, force_rebuild=False):
    """
    Send query to agent; optionally provide PDF for indexing.

    Args:
        query (str): The user's query.
        session_id (str): Session ID for the agent.
        pdf_url (str, optional): PDF URL to build/update index.
        force_rebuild (bool, optional): Force rebuilding the FAISS index.
    """
    parts = [types.Part(text=query)]

    if pdf_url:
        # Include PDF URL and force rebuild flag in the message so the agent can rebuild index
        parts.append(types.Part(text=json.dumps({"pdf_url": pdf_url, "force_rebuild": force_rebuild})))

    content = types.Content(role="user", parts=parts)

    events = runner.run(user_id=USER_ID, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text

    return "Sorry, I couldn't process that request."


## Step 8:  Run Test: Index PDF and Ask First Question

This cell executes the first end-to-end test of the RAG agent.
1.  **Create Session**: It creates a new conversation session using the `session_service`.
2.  **Build Index**: It calls `chat_with_agent` with the query "Build index", a `pdf_url`, and `force_rebuild=True`. This forces the agent to call the `get_context_local` tool, which downloads the sample invoice PDF, processes it, and saves the `local_faiss.index` and `sentences.json` files.
3.  **Query Index**: It calls `chat_with_agent` again, this time with a real question: "what is pdf mainly described about?". The agent will receive this query, decide to use the `get_context_local` tool to search the index it just built, get the relevant context (chunks from the invoice), and then use that context to generate a summary.
4.  **Print Answer**: The final answer is printed.

In [ ]:
# Create a session
session = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)

# 1️⃣ Build index from PDF
response1 = await chat_with_agent(
    query="Build index",
    session_id=session.id,
    pdf_url="https://www.princexml.com/samples/invoice-colorful/invoicesample.pdf",
    force_rebuild=True
)
print("resposnse------------------",response1)

# 2️⃣ Query local vector index
question1="can you provide the overview of the pdf"
response1 = await chat_with_agent(
    query="what is pdf mainly described about?",
    session_id=session.id
)
print("\nQuery:",question1)
print("\nAnswer:",response1)


## Step 9: Run Test: Ask Second Question

This cell runs a second query against the agent within the *same session*. It asks the same question again ("what is pdf mainly described about?"). This demonstrates that:
1.  The local index persists, so the agent doesn't need to rebuild it.
2.  The agent can successfully use the tool to query the existing index.
3.  The agent maintains conversational context (though in this specific query, it's just repeating the action).

In [ ]:
question2="what is pdf mainly described about?"
response2 = await chat_with_agent(
    query=question2,
    session_id=session.id
)
print("\nQuery:",question2)
print("\nAnswer:",response2)